# Hermes
Chat

In [5]:
# Load in libraries
from bs4 import BeautifulSoup
import urllib
import pickle
from operator import itemgetter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
import numpy as np
import math
from copy import copy
from gensim.models import doc2vec, word2vec
from collections import namedtuple

# Scrape and preprocess Facebook Messenger messages

In [9]:
# Read html
page = urllib.urlopen('data/messages.htm').read()
soup = BeautifulSoup(page, "html5lib")

In [10]:
contents_div = soup.body.div.next_sibling
# Get div of room groupchat
groupchat_div = contents_div.div.div

In [11]:
# Create list of all messages in this groupchat
groupchat_messages = []

groupchat_p = groupchat_div.find_all("p")
for p in groupchat_p:
    groupchat_messages.append(p.text)

In [12]:
print "Number of messages:", len(groupchat_messages)
print groupchat_messages[:10]

Number of messages: 9257
[u'leggo', u'hebrew bible leffo', u'looks like culture and belief is all that is left', u'Ez', u'workload is only ~9.4 hours per week we can do this guys', u'graduate level probability', u"stat 210 let's go", u'yah', u'different instructor though', u'according to my.harvard']


We may have to clean this data later, but for now we'll move on

In [13]:
# Create pickle dump of all messages
pickle.dump(groupchat_messages, open('data/groupchat.p', 'wb'))